In [1]:
import numpy as np
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
class EqualizedConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, bias=True):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, bias=bias)
        # initialize weights to N(0,1)
        self.conv.weight.data.normal_(0,1)
        self.conv.bias.data.fill_(0)
    
    def forward(self,x):
        x = self.conv(x)*self.get_scale()
        return x
    
    def get_scale(self):
        size_w = self.conv.weight.size()
        fan_in = size_w[1:].numel()
        return np.sqrt(2/fan_in)

In [3]:
out_ch = 512
in_ch = 512
k = (4,4)
shape = (1,1)
x = torch.randn((1,out_ch,*shape))
print(x.shape)

torch.Size([1, 512, 1, 1])


In [4]:
conv = EqualizedConv2d(out_ch, in_ch, k, padding=3)
conv2 = EqualizedConv2d(in_ch, in_ch, kernel_size=3, padding=1)

In [5]:
conv.conv.weight.std()

tensor(1.0006, grad_fn=<StdBackward0>)

In [6]:
out_x = conv(x)
print(out_x.shape)

torch.Size([1, 512, 4, 4])


In [7]:
conv2(out_x).shape

torch.Size([1, 512, 4, 4])

In [8]:
from PGGAN import PixelWiseNorm

class GblockPGGAN(nn.Module):
    def __init__(self, in_channels, out_channels, first_block=False):
        super().__init__()
        if first_block:
            self.block = nn.Sequential(
                    EqualizedConv2d(in_channels, out_channels, kernel_size=4, padding=3),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    )
        else:
            self.block = nn.Sequential(
                    nn.Upsample(scale_factor=2, mode='nearest'),
                    EqualizedConv2d(in_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2),
                    PixelWiseNorm(),
                    )
    def forward(self, x):
        return self.block(x)    

In [9]:
block1 = GblockPGGAN(in_channels=512, out_channels=512, first_block=True)
block2 = GblockPGGAN(in_channels=512, out_channels=512, first_block=False)
block3 = GblockPGGAN(in_channels=512, out_channels=256, first_block=False)

In [10]:
x = torch.randn((1,512,1,1))
print(block1(x).shape)
print(block2(block1(x)).shape)
print(block3(block2(block1(x))).shape)

torch.Size([1, 512, 4, 4])
torch.Size([1, 512, 8, 8])
torch.Size([1, 256, 16, 16])


In [11]:
to_rgb = EqualizedConv2d(256, 3, 1, padding=0)

In [12]:
to_rgb(block3(block2(block1(x)))).shape

torch.Size([1, 3, 16, 16])

In [13]:
class GeneratorPGGAN(nn.Module):
    def __init__(self, latent_dim=512, out_scale=256):
        super().__init__()

        self.depth = 1
        self.alpha = 1
        self.step_alpha = 0

        self.upscale_blocks = nn.ModuleList()
        self.to_rgb = nn.ModuleList()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        # add first block to get 4x4 image
        self.upscale_blocks.append(GblockPGGAN(in_channels=latent_dim, out_channels=latent_dim, first_block=True))
        self.to_rgb.append( EqualizedConv2d(in_channels=latent_dim, out_channels=3, kernel_size=1, padding=0), )

        start_scale = np.log2(4)
        end_scale = np.log2(out_scale)

        start_scale = int(np.log2(4))
        end_scale   = int(np.log2(out_scale))

        in_ch, out_ch = latent_dim, latent_dim
        for i in range(start_scale+1, end_scale+1):
            in_ch = out_ch
            if i >= 6:
                out_ch = in_ch//2
            else:
                out_ch = in_ch
            self.upscale_blocks.append(GblockPGGAN(in_channels=in_ch, out_channels=out_ch, first_block=False))
            self.to_rgb.append( EqualizedConv2d(in_channels=out_ch, out_channels=3, kernel_size=1, padding=0), )
    
    def forward(self, x):
        # go until second last layer
        for block in self.upscale_blocks[:self.depth-1]:
            x = block(x)
        # compute last layer
        x_prev = x.clone().detach()
        print(f"before {x.shape}")
        x_rgb = self.upscale_blocks[self.depth-1](x)
        print(f"after {x.shape}")
        print(f"are equal? {torch.equal(x,x_prev)}")
        x_rgb = self.to_rgb[self.depth-1](x_rgb)
        #print(f"last layer    x_rgb {x_rgb.shape}   {x.shape}")
        # smooth change
        if self.alpha < 1 and self.depth > 1: # for the first layer cannot be applied
            x_old_rgb = self.upsample(x)
            x_old_rgb = self.to_rgb[self.depth-2](x_old_rgb)
            #print(f"x_old {x_old_rgb.shape}")
            
            x_rgb = self.alpha * x_rgb + (1-self.alpha) * x_old_rgb
        return x_rgb
    
    def increase_net(self, n_iterations):
        self.step_alpha = 1/n_iterations
        self.alpha = 1/n_iterations
        self.depth += 1
    
    def step(self):
        # increase alpha
        self.alpha += self.step_alpha

In [14]:
g = GeneratorPGGAN()

In [15]:
layers = list(dict(g.upscale_blocks[0].block.named_children()).values())
print(layers[0].conv.bias)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.

In [16]:
x = torch.randn((1,512,1,1))

In [17]:
g.depth = 1
g.alpha = 0.5
g.step_alpha = 0.1
for i in range(7):
    out_x = g(x)
    print(f"out tensor {out_x.shape}  alpha {g.alpha}")
    g.depth += 1

before torch.Size([1, 512, 1, 1])
after torch.Size([1, 512, 1, 1])
are equal? True
out tensor torch.Size([1, 3, 4, 4])  alpha 0.5
before torch.Size([1, 512, 4, 4])
after torch.Size([1, 512, 4, 4])
are equal? True
out tensor torch.Size([1, 3, 8, 8])  alpha 0.5
before torch.Size([1, 512, 8, 8])
after torch.Size([1, 512, 8, 8])
are equal? True
out tensor torch.Size([1, 3, 16, 16])  alpha 0.5
before torch.Size([1, 512, 16, 16])
after torch.Size([1, 512, 16, 16])
are equal? True
out tensor torch.Size([1, 3, 32, 32])  alpha 0.5
before torch.Size([1, 512, 32, 32])
after torch.Size([1, 512, 32, 32])
are equal? True
out tensor torch.Size([1, 3, 64, 64])  alpha 0.5
before torch.Size([1, 256, 64, 64])
after torch.Size([1, 256, 64, 64])
are equal? True
out tensor torch.Size([1, 3, 128, 128])  alpha 0.5
before torch.Size([1, 128, 128, 128])
after torch.Size([1, 128, 128, 128])
are equal? True
out tensor torch.Size([1, 3, 256, 256])  alpha 0.5


In [18]:
from_rgb = nn.Sequential( EqualizedConv2d(in_channels=3, out_channels=512, kernel_size=1), nn.LeakyReLU(0.2))

In [19]:
x = torch.randn((1,3,256,256))

In [20]:
from_rgb(x).shape

torch.Size([1, 512, 256, 256])

In [21]:
start_scale = int(np.log2(4))
end_scale = int(np.log2(256))

for i in reversed(range(start_scale, end_scale+1)):
    print(i, 2**i, 512)

8 256 512
7 128 512
6 64 512
5 32 512
4 16 512
3 8 512
2 4 512


In [22]:
start_scale = int(np.log2(4))
end_scale   = int(np.log2(256))
latent_dim = 512
out_ch, in_ch = latent_dim, latent_dim
for i in range(start_scale+1, end_scale+1):
    out_ch = in_ch
    if i >= 6:
        in_ch = out_ch//2
    else:
        in_ch = out_ch
    print(f"in {in_ch} out {out_ch}")

in 512 out 512
in 512 out 512
in 512 out 512
in 256 out 512
in 128 out 256
in 64 out 128


In [23]:
from PGGAN import MinibatchStd

class DblockPGGAN(nn.Module):
    def __init__(self, in_channels, out_channels, last_block=False):
        super().__init__()
        if last_block:
            self.block = nn.Sequential(
                    MinibatchStd(),
                    EqualizedConv2d(in_channels+1, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=4, padding=0),
                    nn.LeakyReLU(0.2, True),
                    nn.Sequential(nn.Flatten(), nn.Linear(out_channels, 1))
                    )
        else:
            self.block = nn.Sequential(
                    EqualizedConv2d(in_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    EqualizedConv2d(out_channels, out_channels, kernel_size=3, padding=1),
                    nn.LeakyReLU(0.2, True),
                    nn.AvgPool2d(kernel_size=2, stride=2) # down sampling
                    )
            
    def forward(self, x):
        return self.block(x) 

In [24]:
class DiscriminatorPGGAN(nn.Module):
    def __init__(self, latent_dim=512, out_scale=256):
        super().__init__()

        self.depth = 1
        self.alpha = 1
        self.step_alpha = 0

        self.downscale_blocks = nn.ModuleList()
        self.from_rgb = nn.ModuleList()
        self.downscale = nn.AvgPool2d(kernel_size=2, stride=2)
        self.from_rgb.append( nn.Sequential( EqualizedConv2d(in_channels=3, out_channels=latent_dim, kernel_size=1), nn.LeakyReLU(0.2) ) )
        # 4x4 -> 1x1
        self.downscale_blocks.append(DblockPGGAN(in_channels=latent_dim, out_channels=1, last_block=True))

        start_scale = int(np.log2(4))
        end_scale   = int(np.log2(out_scale))

        out_ch, in_ch = latent_dim, latent_dim
        for i in range(start_scale+1, end_scale+1):
            out_ch = in_ch
            if i >= 6:
                in_ch = out_ch//2
            else:
                in_ch = out_ch
            self.from_rgb.append( nn.Sequential( EqualizedConv2d(in_channels=3, out_channels=in_ch, kernel_size=1), nn.LeakyReLU(0.2) ) )
            self.downscale_blocks.append(DblockPGGAN(in_channels=in_ch, out_channels=out_ch, last_block=False))

    def forward(self,x_rgb):
        x = self.from_rgb[self.depth-1](x_rgb)
        #print(f"from rgb {x.shape}")
        x = self.downscale_blocks[self.depth-1](x)
        #print(f"down 1 {x.shape}")

        if self.alpha < 1.0 and self.depth > 1:
            x_rgb = self.downscale(x_rgb)
            x_old = self.from_rgb[self.depth-2](x_rgb)
            #print(f"x {x.shape}   x old {x_old.shape}")
            x = self.alpha * x + (1-self.alpha) * x_old 
            self.alpha += self.step_alpha
        i = 2
        for block in reversed(self.downscale_blocks[:self.depth-1]):
            x = block(x)
            #print(f"block {i}   {x.shape}")
            i += 1
        return x  

In [25]:
d = DiscriminatorPGGAN()

In [26]:
latent_dim = 512

In [27]:
d.depth = 1
for i in range(2,9):
    img = torch.randn((1,3,2**i,2**i))
    out_d = d(img)
    print(f"out tensor {out_d.shape}")
    d.depth += 1

/Users/mariobelotti/Documents/Projects/ComputerVision/ComputerVisionApplications/models/PGGAN.py:207: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = torch.std(x, dim=0)


out tensor torch.Size([1, 1])
out tensor torch.Size([1, 1])
out tensor torch.Size([1, 1])
out tensor torch.Size([1, 1])
out tensor torch.Size([1, 1])
out tensor torch.Size([1, 1])
out tensor torch.Size([1, 1])


In [45]:
d.depth = 1
x_hat = torch.randn((32, 3, 4, 4))
x_hat.requires_grad_(True)
y_hat = d(x_hat)

In [50]:
grad = torch.autograd.grad(
    outputs=y_hat,
    inputs=x_hat,
    grad_outputs=torch.ones_like(y_hat),
    create_graph=True,
    retain_graph=True,
    only_inputs = True
)[0]

In [51]:
grad.shape

torch.Size([32, 3, 4, 4])

In [49]:
y_hat.shape

torch.Size([32, 1])